In [2]:
# Add prereq code to set open API key

import os

os.environ["OPENAI_API_KEY"] = os.getenv('OPEN_AI_KEY')
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens = 500)

instruction = """ Andy harvests all the tomatoes from 18 plants that have 7 tomatoes each. If he dries half the
tomatoes and turns a third of the remainder into marinara sauce, how many tomatoes are left? """
print("Original Answer: ")
print(llm.predict(instruction))

Original Answer: 


C:\Users\freda\AppData\Local\Temp\ipykernel_21244\601771148.py:12: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm.predict(instruction))


First, we need to find out how many tomatoes Andy harvested in total:
18 plants * 7 tomatoes = 126 tomatoes

Next, we need to find out how many tomatoes Andy dried:
126 tomatoes / 2 = 63 tomatoes dried

Now we need to find out how many tomatoes are left after drying:
126 tomatoes - 63 tomatoes = 63 tomatoes remaining

Next, we need to find out how many tomatoes Andy turned into marinara sauce:
63 tomatoes / 3 = 21 tomatoes turned into marinara sauce

Finally, we need to find out how many tomatoes are left after making marinara sauce:
63 tomatoes - 21 tomatoes = 42 tomatoes left

Therefore, Andy is left with 42 tomatoes.


Let's see how we can solve this using Chain Of Thought reasoning

In [4]:
question1 = """
Karen harvests all the pears from 20 trees that have 10 pears each. She  throws a third of them away as they are rotten,
and turns a quarter of the remaining ones into jam. How many are left?
"""
answer1 = """
    First, let's calculate how many pears Gloria harvests: it's 20 * 10 = 200. 
    Then, let's calculate how many are rotten: 200 * 1/3 = 66.
    Thus, we know how many are left after she throws a third of them away: 200 - 66 = 134.
    1/4 of the remaining ones are turned into jam, or 134 * 1/4 = 33. Therefore, Karen is left with 134 - 33, or 101 pears
"""
question2 = """
Sergei harvests all the strawberries from 50 plants that have 8 stawberries each. He freezes a quarter of them,
and turns half of the remaining ones into jam. How many are left?
"""
answer2 = """
    First, let's calculate how many strawberries Sergei harvests: it's 50 * 8 = 400. 
    Then, let's calculate how many are frozen: 400 * 1/4 = 100.
    Thus, we know how many are left after he freezes 100 of them: 400 - 100 = 300.
    half of the remaining ones are turned into jam, or 300 * 1/2 = 150. Therefore, Sergei is left with 300 - 150, or 150 pears
"""


example_prompt = PromptTemplate(input_variables=["question", "answer"], 
                                template="{question}\n{answer}")
examples = [ 
    {
        "question": question1,
        "answer": answer1,
    },
    {
        "question": question2,
        "answer": answer2,
    }
]

cot_prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Use these questions and answers to give correct response to the problem below: {input}", 
    input_variables=["input"]
)

cot_text = cot_prompt.format(input=instruction)
print("=== Chain of Thought Prompt ===")
print(cot_text)


=== Chain of Thought Prompt ===

Karen harvests all the pears from 20 trees that have 10 pears each. She  throws a third of them away as they are rotten,
and turns a quarter of the remaining ones into jam. How many are left?


    First, let's calculate how many pears Gloria harvests: it's 20 * 10 = 200. 
    Then, let's calculate how many are rotten: 200 * 1/3 = 66.
    Thus, we know how many are left after she throws a third of them away: 200 - 66 = 134.
    1/4 of the remaining ones are turned into jam, or 134 * 1/4 = 33. Therefore, Karen is left with 134 - 33, or 101 pears



Sergei harvests all the strawberries from 50 plants that have 8 stawberries each. He freezes a quarter of them,
and turns half of the remaining ones into jam. How many are left?


    First, let's calculate how many strawberries Sergei harvests: it's 50 * 8 = 400. 
    Then, let's calculate how many are frozen: 400 * 1/4 = 100.
    Thus, we know how many are left after he freezes 100 of them: 400 - 100 = 300.


In [5]:

print("=== Chain of Thought Answer ===")
print(llm.predict(cot_text))

=== Chain of Thought Answer ===
First, let's calculate how many tomatoes Andy harvests: it's 18 * 7 = 126.
Then, let's calculate how many are dried: 126 * 1/2 = 63.
Thus, we know how many are left after he dries half of them: 126 - 63 = 63.
A third of the remaining ones are turned into marinara sauce, or 63 * 1/3 = 21. Therefore, Andy is left with 63 - 21, or 42 tomatoes.
